In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

sns.set_palette('rocket')
plt.rcParams['image.cmap'] = 'rocket'
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (15, 7)

In [ ]:
ds = pd.read_csv('datasets/cleaned/merged_fx_dataset.csv')

In [ ]:
ds['Date'] = pd.to_datetime(ds['Date'])

In [ ]:
ds.info()

In [ ]:
ds.drop(columns=['Selling', 'Buying'], inplace=True)

In [ ]:
ds.info()

In [ ]:
df_weekly = ds.set_index('Date').resample('W-FRI').last()
df_weekly

In [ ]:
# agg_dict_weekly = {
#     'MidRate': 'last',
#     # 'Buying': 'last', 'Selling': 'last',
#     'MPR': 'last', 'GhInflationRate': 'last', 'BrentOil': 'last',
#     'Cocoa': 'last', 'Gold': 'last', 'GhInterestRate': 'last',
#     'NetForeignAssets': 'last', 'NIR': 'last', 'Imports': 'sum', 'Exports': 'sum',
#     'USInflationRate': 'last', 'USInterestRate': 'last'
# }

# df_weekly = ds.set_index('Date').resample('W-FRI')

In [ ]:
df_weekly.info()

In [ ]:
selected_features = [
    'MidRate', 'GhInflationRate', 'USInflationRate', 'GhInterestRate', 'USInterestRate',
    'BrentOil', 'Cocoa', 'Gold', 'Imports', 'Exports'
]

In [ ]:
df_weekly = df_weekly[selected_features].copy()

In [ ]:
df_weekly.dropna(inplace=True)

print("Weekly Data Head:")
df_weekly.head()

In [ ]:
df_weekly.info()

#### Data Preprocessing

In [ ]:
# For now, we'll just define the function.
def scale_data(train, test):
    scaler = MinMaxScaler(feature_range=(0, 1))
    train_scaled = scaler.fit_transform(train)
    test_scaled = scaler.transform(test)
    return scaler, train_scaled, test_scaled

In [ ]:
# Function to create sequences for LSTM
def create_sequences(data, lookback_period):
    X, y = [], []
    for i in range(len(data) - lookback_period):
        X.append(data[i:(i + lookback_period), :])
        # The target is the 'MidRate' of the next week (which is at index 0 of our features)
        y.append(data[i + lookback_period, 0])
    return np.array(X), np.array(y)

In [35]:
LOOKBACK_WEEKS = 20

#### Build the LSTM Model

In [36]:
# def build_lstm_model(input_shape):
#     model = Sequential()
#     # From paper: 34 nodes in LSTM layer
#     model.add(LSTM(34, activation='tanh', input_shape=input_shape))
#     # From paper: 1 node in output layer
#     model.add(Dense(1))
#     # From paper: Adam optimizer with lr=0.002
#     optimizer = tf.keras.optimizers.Adam(learning_rate=0.002)
#     model.compile(optimizer=optimizer, loss='mean_squared_error')
#     return model


# --- 5. Build a MORE STABLE LSTM Model Architecture ---
def build_lstm_model(input_shape):
    model = Sequential()
    # Reduce units and add gradient clipping to the optimizer
    model.add(LSTM(34, activation='relu', input_shape=input_shape)) # Reduced from 34 to 16 units
    model.add(Dense(1))

    # Use a lower learning rate and add gradient clipping
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005, clipnorm=1.0) # Lower LR and added clipping
    model.compile(optimizer=optimizer, loss='mean_squared_error')
    return model

# Re-define the input shape (it remains the same)
input_shape = (LOOKBACK_WEEKS, len(selected_features))
stable_lstm_model = build_lstm_model(input_shape)
stable_lstm_model.summary()

/Users/dela/Developer/thesis/.venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_4 (LSTM)                   │ (None, 34)             │         6,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │            35 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,155 (24.04 KB)

 Trainable params: 6,155 (24.04 KB)

 Non-trainable params: 0 (0.00 B)

In [37]:
# Define the input shape (lookback weeks, number of features)
input_shape = (LOOKBACK_WEEKS, len(selected_features))
lstm_model = build_lstm_model(input_shape)
lstm_model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_5 (LSTM)                   │ (None, 34)             │         6,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            35 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,155 (24.04 KB)

 Trainable params: 6,155 (24.04 KB)

 Non-trainable params: 0 (0.00 B)

##### Walk-Forward Validation

In [30]:
df_weekly.index.min()

Timestamp('2008-01-04 00:00:00')

In [40]:
# --- 6. Walk-Forward Validation (WITH DEBUGGING) ---
print("\nStarting LSTM Walk-Forward Validation...")
results_lstm = []

# We need a bit more data to start, so let's start the test set a bit later
start_test_date = df_weekly.index.min() + pd.DateOffset(years=2)

# Loop through time, testing on 6-month periods
for end_test_date in pd.date_range(start_test_date, df_weekly.index.max(), freq='6M'):
    start_test_date = end_test_date - pd.DateOffset(months=6)

    # Define train and test sets for this fold
    train = df_weekly[(df_weekly.index < start_test_date)]
    test = df_weekly[(df_weekly.index >= start_test_date) & (df_weekly.index < end_test_date)]

    print(f"\n--- Iteration for test period {start_test_date.date()} - {end_test_date.date()} ---")
    print(f"Initial train size: {len(train)}, test size: {len(test)}")

    if len(train) < LOOKBACK_WEEKS * 2 or len(test) < LOOKBACK_WEEKS:
        print("SKIPPED: Not enough data in train/test split.")
        continue

    # Scale data
    scaler, train_scaled, test_scaled = scale_data(train, test)

    # Create sequences
    X_train, y_train = create_sequences(train_scaled, LOOKBACK_WEEKS)
    X_test, y_test = create_sequences(test_scaled, LOOKBACK_WEEKS)

    print(f"Sequence shapes - X_train: {X_train.shape}, y_train: {y_train.shape}, X_test: {X_test.shape}, y_test: {y_test.shape}")

    if len(X_train) == 0 or len(X_test) == 0:
        print("SKIPPED: Not enough data to create sequences.")
        continue

    # Build and train model for this fold
    model = build_lstm_model(input_shape=(LOOKBACK_WEEKS, len(selected_features)))
    # From paper: batch_size=128, epochs=100
    model.fit(X_train, y_train, epochs=100, batch_size=128, verbose=0)


    # Make predictions
    y_pred_scaled = model.predict(X_test, verbose=0)

    # --- SIMPLIFIED ROBUST EVALUATION ---
    # Calculate R² directly on the scaled data. This avoids all inverse transform issues.
    # Note: y_test is already scaled.
    r2_model = r2_score(y_test, y_pred_scaled)

    # For the naive baseline on scaled data, we predict the last value of the sequence
    # The last value of each X_test sequence is X_test[:, -1, 0]
    naive_pred_scaled = X_test[:, -1, 0]
    r2_naive = r2_score(y_test, naive_pred_scaled)

    print(
        f"SUCCESS: Scaled R² - Model: {r2_model:.4f}, Naive: {r2_naive:.4f}")

    # We still need to store the results. Let's store the scaled scores for now.
    results_lstm.append({
        'test_start': start_test_date,
        'r2_model': r2_model,  # Storing the scaled R²
        'r2_naive': r2_naive  # Storing the scaled R²
    })
    print(f"SUCCESS: Appended result. Model R²={r2_model:.4f}, Naive R²={r2_naive:.4f}")


# --- 7. Final Analysis (WITH SAFETY CHECK) ---
print("\n--- LSTM Walk-Forward Results ---")

# Add a safety check before creating the DataFrame
if not results_lstm:
    print("WARNING: The results list is empty. The loop did not complete any successful iterations.")
    print("This is likely due to insufficient data after resampling to weekly.")
    print("Please check the debug output above to see why the loop was skipped.")
else:
    results_lstm_df = pd.DataFrame(results_lstm)
    print(results_lstm_df)

    avg_r2_model_lstm = results_lstm_df['r2_model'].mean()
    avg_r2_naive_lstm = results_lstm_df['r2_naive'].mean()

    print(f"\nAverage LSTM Model R²: {avg_r2_model_lstm:.4f}")
    print(f"Average Naive R² (Weekly): {avg_r2_naive_lstm:.4f}")

    if avg_r2_model_lstm > avg_r2_naive_lstm:
        print("\nSUCCESS: The LSTM model (from paper methodology) provided a meaningful edge.")
    else:
        print("\nFAILURE: The LSTM model did not beat the naive baseline.")


Starting LSTM Walk-Forward Validation...

--- Iteration for test period 2009-07-31 - 2010-01-31 ---
Initial train size: 82, test size: 27
Sequence shapes - X_train: (62, 20, 10), y_train: (62,), X_test: (7, 20, 10), y_test: (7,)


/var/folders/pq/lv4328kx4fjg40sltdfjtrcm0000gn/T/ipykernel_53711/1335166170.py:9: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  for end_test_date in pd.date_range(start_test_date, df_weekly.index.max(), freq='6M'):
/Users/dela/Developer/thesis/.venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


SUCCESS: Scaled R² - Model: -19145.1940, Naive: 0.5489
SUCCESS: Appended result. Model R²=-19145.1940, Naive R²=0.5489

--- Iteration for test period 2010-01-31 - 2010-07-31 ---
Initial train size: 109, test size: 26
Sequence shapes - X_train: (89, 20, 10), y_train: (89,), X_test: (6, 20, 10), y_test: (6,)


/Users/dela/Developer/thesis/.venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


SUCCESS: Scaled R² - Model: -8256.0877, Naive: 0.3737
SUCCESS: Appended result. Model R²=-8256.0877, Naive R²=0.3737

--- Iteration for test period 2010-07-31 - 2011-01-31 ---
Initial train size: 135, test size: 26
Sequence shapes - X_train: (115, 20, 10), y_train: (115,), X_test: (6, 20, 10), y_test: (6,)


...

SUCCESS: Scaled R² - Model: -243902.2345, Naive: 0.4255
SUCCESS: Appended result. Model R²=-243902.2345, Naive R²=0.4255

--- Iteration for test period 2023-07-31 - 2024-01-31 ---
Initial train size: 813, test size: 26
Sequence shapes - X_train: (793, 20, 10), y_train: (793,), X_test: (6, 20, 10), y_test: (6,)


/Users/dela/Developer/thesis/.venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


SUCCESS: Scaled R² - Model: -5356.8606, Naive: 0.1860
SUCCESS: Appended result. Model R²=-5356.8606, Naive R²=0.1860

--- Iteration for test period 2024-01-31 - 2024-07-31 ---
Initial train size: 839, test size: 26
Sequence shapes - X_train: (819, 20, 10), y_train: (819,), X_test: (6, 20, 10), y_test: (6,)


/Users/dela/Developer/thesis/.venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


SUCCESS: Scaled R² - Model: -37.1975, Naive: 0.4185
SUCCESS: Appended result. Model R²=-37.1975, Naive R²=0.4185

--- LSTM Walk-Forward Results ---
   test_start      r2_model  r2_naive
0  2009-07-31 -1.914519e+04  0.548894
1  2010-01-31 -8.256088e+03  0.373666
2  2010-07-31 -5.327260e+02  0.417811
3  2011-01-31 -8.903334e+06 -0.230000
4  2011-07-31 -1.657090e+01  0.071861
5  2012-01-31 -2.978637e+02  0.531181
6  2012-07-31 -3.500376e+02  0.436406
7  2013-01-31 -6.192464e+01 -2.114160
8  2013-07-31 -2.758647e+00  0.701956
9  2014-01-31 -7.428899e+02 -3.015919
10 2014-07-31 -1.770758e+01  0.736994
11 2015-01-31 -1.248707e+00  0.501974
12 2015-07-31 -3.669318e+02  0.627432
13 2016-01-31 -6.338915e+01  0.434405
14 2016-07-31 -2.067063e+02  0.288004
15 2017-01-31 -6.828189e+03 -0.526350
16 2017-07-31 -1.430556e+03  0.656210
17 2018-01-31 -4.822594e+01  0.483593
18 2018-07-31 -1.714272e+01  0.678762
19 2019-01-31 -2.197176e+03 -4.369159
20 2019-07-31 -6.850283e+02  0.180295
21 2020-01-31 -3